# DSCI 575 - Advanced Machine Learning

# Lab 2: Markov Models

## Table of contents
- [Submission guidelines](#sg)
- [Learning outcomes](#lo)
- [Exercise 1: Warm-up](#wu)
- [Exercise 2: Character-based Markov model of language](#mmch)
- [Exercise 3: Stationary distribution and other fun stuff](#sd)
- [Exercise 4: Word-based Markov model of language](#mmw)

## Submission guidelines <a name="sg"></a>

#### Tidy submission
rubric={mechanics:2}
- To submit this assignment, submit this jupyter notebook with your answers embedded.
- Be sure to follow the [general lab instructions](https://ubc-mds.github.io/resources_pages/general_lab_instructions/).
- Use proper English, spelling, and grammar throughout your submission.

#### Code quality and writing
- These rubrics will be assessed on a question-by-question basis and are included in individual question rubrics below where appropriate.
- See the [quality rubric](https://github.com/UBC-MDS/public/blob/master/rubric/rubric_quality.md) and [writing rubric](https://github.com/UBC-MDS/public/blob/master/rubric/rubric_writing.md) as a guide to what we are looking for.
- Refer to [Python PEP 8 Style Guide](https://www.python.org/dev/peps/pep-0008/) for coding style.

In [ ]:
import os
import sys
import numpy as np
import numpy.random as npr
import re
from collections import defaultdict
from collections import Counter

import pandas as pd
from urllib.request import urlopen

## Learning outcomes <a name="lo"></a>

After working on this lab, you will be able to

- Build a character-based Markov model of language. 
- Explain states, state space, and transition matrix in Markov models of language. 
- Explain and calculate stationary distribution over states in Markov models of language. 
- Calculate the probability of patterns using Markov models of language. 
- Build a word-based Markov model of language and generate text using this model. 

## Exercise 1: Warm-up <a name="wu"></a>

This exercise will get you thinking about how to generate text using Markov models. You will build a simple Markov model with a tiny corpus, where the current letter only depends upon the previous letter ($n=1$).  

Below I am providing you some starter code to create bigram frequency counts of letters in a tiny corpus with 6 words. Our goal is to build a Markov model with this corpus, where the set of states is the unique letters in the corpus.  

Note: Recall that in DSCI 512 lab1, Exercise 4, you implemented a Markov model of language and generated text  (without actually knowing the details about the model). You pre-computed the frequencies for every possible $n$ gram as follows.  

In [ ]:
### BEGIN STARTER CODE
corpus = 'to be or not to be'
n = 1
circ_corpus = corpus + corpus[:n]
frequencies = defaultdict(Counter)
for i in range(len(circ_corpus) - n):
    frequencies[circ_corpus[i:i + n]][circ_corpus[i + n]] += 1
frequencies    
### END STARTER CODE

### 1(a) Visualizing character bigram counts as a co-occurrence matrix
rubric={viz:2} 

Now let's try to visualize the bigram frequency counts calculated above as a pandas DataFrame. 

Your task:

- Show the bigram frequencies in `frequencies` variable above as a pandas DataFrame, sorting the column and row labels alphabetically, where s
    * column labels and row indices are unique characters in the corpus, and 
    * the value in each cell $a_{ij}$ represents how often the character $j$ is followed by character $i$ in our corpus.    

In [ ]:
### YOUR ANSWER HERE

### 1(b) Transition matrix
rubric={accuracy:3,viz=1}

Given the frequencies in 1(a), let's compute the transition matrix (i.e., the conditional probability distribution for every possible $n$ gram). 

1. Normalize each row in the above matrix so that each row sums to 1.0.
2. Visualize this normalized matrix as a pandas DataFrame. Now you have the transition matrix of your Markov model! 
3. How many unique states are there in your Markov model? 

Recall that the transition matrix $T$ is a square matrix and the number of rows/columns is equal to the number of states. Each row is a discrete probability distribution summing to 1. The element $T_{ij}$ is the probability of transitioning from state $i$ to state $j$. 

### YOUR ANSWER HERE

In [ ]:
### YOUR ANSWER HERE

In [ ]:
### YOUR ANSWER HERE

In [ ]:
### YOUR ANSWER HERE 

### 1(c) Probability of sequences
rubric={reasoning:2}

Suppose the probability of starting a sequence with letter $b$ is $0.4$.Given the transition matrix in 1(b), what are the probabilities of the following sequences?
    * be or
    * beet

### YOUR ANSWER HERE

### 1(d) Generate sequences
rubric={reasoning:2}

- Given the transition matrix above, explain in words how would you generate a sequence of length `seq_len` given the start symbol `t`. You don't have to, but you are welcome to write the code for this. 

### YOUR ANSWER HERE

In [ ]:
### YOUR ANSWER HERE

## Exercise 2: Character-based Markov model of language <a name="mmch"></a>

Recall Exercise 4: Markov Model of language from DSCI 512 lab1. In this exercise, you wrote a class `MarkovModel`
 to `fit` an n-gram Markov model and generated text. Now that you know what Markov models are, we will start with what you did in DSCI 512 lab1 and build on it. 

The starter code below uses the hyperparameter $n$, where our "state" of the Markov chain is the last $n$ characters of a give string. In Exercise 1, we worked with $n = 1$ and our "state" of the Markov chain was a single character and each character was dependent on the last one character. When $n=3$, it means that the probability distribution over the _next character_ only depends on the _preceding 3 characters_. We use the name [_n-gram_](https://en.wikipedia.org/wiki/N-gram) for a sequence of $n$ characters.

We train our model from data by recording every occurrence of every $n$-gram and, in each case, recording what the next letter is. Then, for each $n$-gram, we normalize these counts into probabilities just like we did with naive Bayes. The `fit` function below implements these steps.

To generate a new sequence, we start with some initial seed of length $n$ (here, we will just use the first $n$ characters in the training text, which are saved at the end of the `fit` function). Then, for the current $n$-gram we will look up the probability distribution over next characters and sample a random character according to this distribution.

Attribution: assignment adapted with permission from Princeton COS 126, [_Markov Model of Natural Language_]( http://www.cs.princeton.edu/courses/archive/fall15/cos126/assignments/markov.html). Original assignment was developed by Bob Sedgewick and Kevin Wayne. If you are interested in more background info, you can take a look at the original version. The original paper by Shannon, [A Mathematical Theory of Communication](http://math.harvard.edu/~ctm/home/text/others/shannon/entropy/entropy.pdf), essentially created the field of information theory and is, in my opinion, one of the best scientific papers ever written (in terms of both impact and readability).  

In [ ]:
### BEGIN STARTER CODE
class MarkovModel():
    def __init__(self, n):
        """
        Arguments:
        n -- (int) 
            the size of the ngram
        """
        self.n = n
    
    def fit(self, text):
        """        
        Fit a Markov chain and create a transition matrix.
        Arguments:
        text -- (str) 
            text to fit the Markov chain         
        """
        # make text circular so markov chain doesn't get stuck
        circ_text = text + text[:self.n] 
        
        # count the number of occurrences of each letter following a given n-gram
        frequencies = defaultdict(Counter)
        for i in range(len(text)):
            frequencies[circ_text[i:i+self.n]][circ_text[i+self.n]] += 1.0
            
        # normalize the frequencies into probabilities (separately for each n-gram)
        self.probabilities_ = defaultdict(dict)
        for ngram, counts in frequencies.items():            
            self.probabilities_[ngram]["symbols"] = list(counts.keys())            
            probs = np.array(list(counts.values()))
            probs /= np.sum(probs)
            self.probabilities_[ngram]["probs"] = probs 
        
        # store the firstnk characters of the training text, as we will use these
        # to seed our `generate` function
        self.starting_chars = text[:self.n]
        self.frequencies_ = frequencies # you never know when this might come in handy
   
    def generate(self, seq_len):
        '''
        Using self.starting_chars, generate a sequence of length seq_len 
        using the transition matrix created in the fit method.

        Arguments:
        seq_len -- (int) 
            the desired length of the sequence

        Returns:
        (str) The generated sequence
        '''                
        # YOUR CODE HERE
        # BEGIN SOLUTION 
        s = self.starting_chars
        while len(s) < seq_len:
            probs = self.probabilities_[s[-self.n:]]
            s += npr.choice(probs["symbols"], p=probs["probs"])
        return s    
### END STARTER CODE    

In [ ]:
### BEGIN STARTER CODE
# Let's test our code. 
data_url = 'http://www.gutenberg.org/files/2591/2591-0.txt'
corpus = urlopen(data_url).read().decode("utf-8")
corpus = corpus[2000:]
model = MarkovModel(n=5)
model.fit(corpus)
print(model.generate(500))
### END STARTER CODE

As $n$ gets larger the output makes more sense. Around $n=5$ it looks like English. Around $n=15$ it look like it's memorizing the training set. 


### 2(a) "Circular" version of the text
rubric={reasoning:2}

Why do we need to use a "circular" version of the text in the `fit` function? What could go wrong if we didn't do that, and instead used the original `text` (insetad of `circ_text`) and made the loop 
```
for i in range(len(text)-self.n):
```
which allow `fit` to run without crashing?

Hint: the problem would arise during `generate`, not during `fit`.



### YOUR ANSWER HERE

### 2(b) Smoothing
rubric={reasoning:2}

The description above mentioned a connection to naive Bayes. When discussing naive Bayes we briefly mentioned [Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing). Is there something analogous to Laplace smoothing that we could do here? If so, describe what it would entail and what effect it would have. (You don't have to implement it.)

### YOUR ANSWER HERE

### 2(c) States, state space, and transition matrix
rubric={reasoning:10}

Let's consider the Markov chain interpretation of what we've just done. Let's define a state as the previous $n$ characters "emitted" by our chain. Let's assume our vocabulary size (the number of possible characters) is $V$; for example, $V=26$ if we were only using lowercase letters. We can compute this in our case:

In [ ]:
print("Vocabulary size = %d" % len(np.unique(list(corpus))))

But let's just stick with a general $V$ for now. Call the number of possible states $S$. Let's consider the _transition matrix_ $T$ of this Markov chain. The transition matrix is a square matrix and the number of rows/columns is equal to the number of states, so in our case it is $S \times S$. Each row is a discrete probability distribution summing to 1. The element $T_{ij}$ is the probability of transitioning from state $i$ to state $j$. 

Answer the following questions:

1. What is the number of possible states, $S$, in terms of $V$ and $n$? (You might not encounter all these states in practice, if not all $n$-grams appear in the training data, but let's ignore that for now.)
2. Give two examples states in the state space for $n=3$.
3. In our application above when $n>1$ _not all transitions are possible_. This means $T$ will contain a bunch of zeros. Why?
4. Again for $n>1$ what is the maximum number of nonzero elements that $T$ could have? Answer in terms of $V$ and $n$. 
5. Is the state space discrete or continuous?

### YOUR ANSWER HERE

### Exercise 3: Stationary distribution and other fun stuff <a name="sd"></a>

The code below computes the transition matrix for you, for the Shakespeare data with $n=1$. Consider this transition matrix for the next few questions.

In [ ]:
### BEGIN STARTER CODE 
data_url = 'http://www.gutenberg.org/files/100/100-0.txt'
shakespeare_text = urlopen(data_url).read().decode("utf-8")
shakespeare_text = shakespeare_text[4000:]
### END STARTER CODE 

In [ ]:
### BEGIN STARTER CODE 
states = np.unique(list(shakespeare_text))
print("States:", states)
S = len(np.unique(list(shakespeare_text)))
print("Number of states:", S)

model = MarkovModel(n=1)
model.fit(shakespeare_text)

# implementation note: since len(model.probabilities_[state]["probs"]) might not equal S
# for all letters, we need to be careful and do a reverse-lookup for the actual letters
# the rest of the transition probabilities are just zero (if they don't appear)
lookup = dict(zip(states, list(np.arange(S, dtype=int))))

T = np.zeros((S,S)) # transition matrix
for i,state in enumerate(states):
    for j, letter in enumerate(model.probabilities_[state]["symbols"]):
        T[i, lookup[letter]] = model.probabilities_[state]["probs"][j]
# print(T)
print("Number of nonzero transition probabilities: %d/%d" % (np.sum(T>0), T.size))
### END STARTER CODE 

### 3(a) Stationary distribution conditions 
rubric={reasoning:4}

Under mild assumptions, a Markov chain has a _stationary distribution_ which is the probability of finding yourself in each state after the chain is run for a long time. These assumptions are:

- "Irreducible" (doesn’t get stuck in part of the graph)
- "Aperiodic" (doesn’t keep repeating same sequence).

Give a short explanation for why our Markov chain satisfies these assumptions.

### YOUR ANSWER HERE

### 3(b) Stationary distribution for the `shakespeare_text`
rubric={accuracy:5}

It's not true in general but in this particular case we actually already know the stationary distribution -- it is just the relative frequency that these $n$-grams occur in the training data (you are welcome to think about this deeply, but you don't have to). (Recall that we are assuming n=1 here.)

Your tasks: 

1. Calculate the stationary distribution for the `shakespeare_text` by calculating the relative frequencies for each n-gram (i.e., state) in the corpus. In particular, for each n-gram or state $s_i$ you can calculate $P(s_i)$ by counting the number of occurrences of the state $s_i$ in the corpus and divide it by the total number of n-grams in the corpus, similar to how Andrey Markov calculated stationary distribution for vowels and consonants in his corpus.
2. Show empirically that this is a stationary distribution. 

In [ ]:
### YOUR ANSWER HERE

### (optional) 3(c) Stationary distribution using eigenvalue decomposition
rubric={reasoning:1}

You can compute the stationary distribution in another ways. 

1. Calculate stationary distribution using eigenvalue decomposition of the transition matrix. 
2. Compare your result from part 3(b) with the result below. Do they agree?

In [ ]:
### YOUR ANSWER HERE

### 3(d) Finding probability of occurrences of patterns 
rubric={reasoning:6}

Let's consider the conditional probability that a lower case vowel comes 3 characters later given that the current letter is "a". In other words, we're searching for the pattern `axxv` where `v` is a lower case vowel (defined as a,e,i,o,u,y) and `x` is any character and `a` is literally "a". 

Let's use $n=1$.

1. It turns out we can estimate this probability directly from the transition matrix. While $T$ gives us the probabilities one step ahead, $T\times T$ gives us the probabilities 2 steps ahead, etc. (If you want to think about this, you should be able to convince yourself it's true by contemplating what matrix multiplication really means, but this is optional.) So, taking $T^3$ gives us the transition probabilities for 3 steps later. Compute $T^3$ and find the estimated probability that we're looking for. Note: you should NOT use `T**3`, as that is elementwise exponentiation rather than a matrix power. You can get $T^3$ using `numpy.linalg.matrix_power(T,3)` or just `T@T@T`.
2. We could also estimate this probability directly from the data, by just seeing how frequently this pattern occurs. Do this. How well does it match your above results? What does this have to do with the quality of our Markov assumption?
3. What if we increased $n$ and repeated part (1), would you expect to get closer to the answer from part (2)? You are welcome to try this but you don't have to - the goal is just to think about it.

In [ ]:
### YOUR ANSWER HERE

In [ ]:
### YOUR ANSWER HERE

In [ ]:
### YOUR ANSWER HERE

A close match. This means the Markov assumption is working reasonably well for this problem. It doesn't mean that the output looks like text though! It just means the assumptions in the Markov chain are doing a reasonably good job of capturing this particular conditional probability.

### YOUR ANSWER HERE

## Exercise 4: Markov model of language with words <a name="mmw"></a>

In this exercise we'll continue with the Markov model of language, but we'll work with _words_ instead of _characters_. The `MarkovModel` code stays the same. Just remember that now we are dealing with words and not characters. 

When we say $n$-gram we're now referring to the last $n$ words, not the last $n$ characters. In general, the term $n$-gram can refer to characters or word; see the [Wikipedia article](https://en.wikipedia.org/wiki/N-gram).

One concern with words is that, in a sense, we have less data to work with. For example with characters and $n=3$ we have a lot of examples of seeing the character combination "per" and checking what character follows it, but with words even with $n=1$ (the smallest nontrivial value of $n$) we might only have one example of the word "persuade" in our corpus. You can imagine that the number of "training examples" only gets smaller if $n>1$. This is something to keep in mind when deciding what values of $n$ might be reasonable.

Another issue is how to deal with punctuation. How do we define a word exactly? We will use the standard convention that punctuation characters are considered separate words. To accomplish this preprocessing, we will use the popular python package [NLTK](http://www.nltk.org/) (Natural Language ToolKit), which we have seen in class. You should be able to install it with

```
conda install nltk
```
Afterwards, you'll need to install the NLTK data files (these are often natural-language-specific). You can grab the tokenizer we need with
```
python -m nltk.downloader 'punkt'
```
If you end up using NLTK for other stuff later on, you can download the rest of the data. 

### 4(a) Word-based n-gram language model
rubric={accuracy:4,quality:2,reasoning:2}

The first step is to break apart the text string into words. NLTK's `word_tokenize` will turn our text into a lists of "tokens" which we will use as our language "atoms". This tokenizing process removes whitespace. So, in `generate`, you should add a space character back in between tokens. This won't look quite right, but it's good enough for now.

Your tasks:
1. Use NLTK for word tokenization. 
2. Implement a word-based Markov model of language.
3. Compare it to your character-based model. 

Optional note: only the `generate` function needs to be rewritten, and, even there, the required modifications are not large. I don't think we talked about class inheritance in DSCI 524 (right?), but it's a handy trick. The line
```
class MarkovModelWords(MarkovModel):
```
defines a new class called `MarkovModelWords` that you can think of as an offspring of `MarkovModel`. It "inherits" all the functions from `MarkovModel` unless we explicitly overwrite them. So, if you implement `generate` in the class `MarkovModelWords`, the old `generate` funtion will be overwritten but all the other functions will persist. 

But, if you're not comfortable with inheritance, or just don't feel like it, you're completely welcome to just modify the above code and not use inheritance.

Another note: the way `MarkovModel` is implemented, the patterns we're conditioning on (the last $n$ characters/words) are used as keys in python dictionaries. For characters that was easy because we could just use a string as a key. However, for words with $n>1$ we now have a collection of words, and we want to use this as a dict key. `nltk.tokenize.word_tokenize` outputs a list, which can't be used as a key. I've casted the list to a tuple in the code below, because a tuple _can_ be used as a dict key (if you case: I believe this is because it's immutable whereas a list is not). This tiny change allows reuse of the old code.

In [ ]:
### BEGIN STARTER CODE
import nltk
from nltk.tokenize import word_tokenize
text_tok = word_tokenize(shakespeare_text)      
text_tok = tuple(text_tok) 
### END STARTER CODE

In [ ]:
### YOUR ANSWER HERE

### 4(b) Dealing with punctuation
rubric={accuracy:4}

If you've generated text from part (a), you probably noticed that the way whitespace is handled is not ideal. In particular, whitespace is added after every token, including right before punctuation. 

1. Modify your code to fix this and show a generated sequence. You can access a list of punctuation characters with `string.punctuation` (and probably somewhere in NLTK as well).
2. Show some results for different values of $n$. Note your observations on how the value of `n` affects the quality of the generated text and time taken to generate the text. 

In [ ]:
### YOUR ANSWER HERE

### (optional) 4(c) other improvements
rubric={reasoning:1}

Are there other improvements you'd like to make to your word-based Markov model so that the generated text looks like proper English? Feel free to either implement them or just describe what you might want to do.